![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/annotation/text/english/sentence-embeddings/BGEEmbeddings.ipynb)

## Colab Setup

In [1]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.3.1
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.8/564.8 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 19.1 MB/s eta 0:00:00


# Download BGEEmbeddings Model and Create Spark NLP Pipeline

Lets create a Spark NLP pipeline with the following stages:

In [2]:
import sparknlp
from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd

# for GPU training >> sparknlp.start(gpu = True)
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

Spark NLP version 5.3.1
Apache Spark version: 3.2.3


In [3]:
BGEEmbeddings

sparknlp.annotator.embeddings.bge_embeddings.BGEEmbeddings

In [4]:
document = DocumentAssembler() \
            .setInputCol("text") \
            .setOutputCol("document")

embeddings = BGEEmbeddings.pretrained("bge_base", "en") \
            .setInputCols("document") \
            .setOutputCol("embeddings")

pipeline = Pipeline().setStages([
     document,
     embeddings
 ])

bge_base download started this may take some time.
Approximate size to download 246.7 MB
[OK!]


Lets create a dataframe with some queries and passages to be used as input for the pipeline.

In [15]:
text = "John Snow (15 March 1813 – 16 June 1858) was an English physician and a leader in the development of anaesthesia and medical hygiene. He is also considered one of the founders of modern epidemiology."
data = spark.createDataFrame([[text]]).toDF("text")

Run the pipeline and get the embeddings.

In [16]:
results = pipeline.fit(data).transform(data)
results.select("embeddings.embeddings").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------